In [ ]:
import os
import numpy as np
import pandas
import re
import urllib
from bs4 import BeautifulSoup as bs

debug = 2
print('ok')

In [ ]:
#songonly_signs = re.compile('(?i)^[\s\-\._]*(br\-|dis|cd|artist|unknown|sample|side|track|tit[le|el]|t[\d.])[\s\-\._]*')


os.chdir('/home/idies/workspace/raddick_folksonomy/data')
music_df = pandas.read_csv('cleaned_audio_data.csv', index_col='ID')
music_df = music_df.rename(columns={'Source': 'source', 'Sample File Name': 'filename'})

# Everything after the final period is the filename
if (debug >= 2):
    print('Parsing filename into stem and filetype...')
music_df['filetype'] = music_df['filename'].apply(lambda x: x.split('.')[-1])

# Delete stray .cue file
if (debug >= 2):
    print('Deleting stray .cue file...')
music_df = music_df.drop(music_df.index[music_df['filetype'] == 'cue'], axis=0)

# Everything before the final period is the stem
music_df['stem'] = music_df['filename'].apply(lambda x: x[0:len(x)-len(x.split('.')[-1])-1])

music_df['songonly'] = False

cd_like = re.compile('(?i)^[\s\-\._]*(br\-|dis|cd|sample|side|track|tit[le|el]|t[\d.])[\s\-\._]*')
onlynumbers = re.compile('^[0-9\.]+$')
intro_like = re.compile('(?i)^\d+[\s\.\-_\(\[]+\w*(intro|arriv|crowd|audience|stage|applause|tuning|announce|welcome|banter)')
unknown_like = re.compile('(?i)^\d*[\s\.\-_\(\[]*(unknown|artist|title)')
song_track_like = re.compile('(?i)^(d|\d)+[\s\.\-_\(\[\w]*(track)[\s\-\._]*\d+')
otherpatternfinder = re.compile('(?i)\d+\s*dr\d+')

music_df.loc[music_df['stem'].apply(lambda x: re.search(cd_like, x)).notnull(), 'songonly'] = True
music_df.loc[music_df['stem'].apply(lambda x: re.search(onlynumbers, x)).notnull(), 'songonly'] = True
music_df.loc[music_df['stem'].apply(lambda x: re.search(intro_like, x)).notnull(), 'songonly'] = True
music_df.loc[music_df['stem'].apply(lambda x: re.search(unknown_like, x)).notnull(), 'songonly'] = True
music_df.loc[music_df['stem'].apply(lambda x: re.search(song_track_like, x)).notnull(), 'songonly'] = True
music_df.loc[music_df['stem'].apply(lambda x: re.search(otherpatternfinder, x)).notnull(), 'songonly'] = True


zeros_df = music_df[music_df['songonly']]
music_df = music_df[~music_df['songonly']]
print('zeros_df has ', len(zeros_df))
print('music_df has ', len(music_df))

music_df = music_df.drop('songonly', axis=1)

music_df_bk = music_df
print('ok')

#music_df['stem'][
#    (music_df['songonly']) 
#].sort_values()[125:]

In [ ]:
music_df = music_df_bk

if (debug >= 2):
    print('Parsing stem into band and post-band...')
music_df.loc[music_df['stem'].apply(lambda x: re.search('\d', x)).notnull(), 'band_abbrev_autoparsed'] = music_df['stem'][music_df['stem'].apply(lambda x: re.search('\d', x)).notnull()].apply(lambda x: x[0:re.search('\d',x).start()])
music_df['postbandstuff'] = music_df['stem'][music_df['stem'].apply(lambda x: re.search('\d+', x)).notnull()].apply(lambda x: x[re.search('\d', x).start():len(x)])

# If no digits found, everything is part of the band_abbrev_autoparsed
if (debug >= 2):
    print('Fixing band values for stems with no digits...')

music_df['band_abbrev_autoparsed']  = music_df['band_abbrev_autoparsed'].fillna(music_df['stem'])
music_df['postbandstuff']  = music_df['postbandstuff'].fillna('')

# Fix the band U2
if (debug >= 2):
    print('Fixing band values for U2...')


music_df.loc[
    music_df['band_abbrev_autoparsed'].apply(lambda x: x.lower() == 'u')
    , 'band_abbrev_autoparsed'
] = music_df['stem'][
    music_df['band_abbrev_autoparsed'].apply(lambda x: x.lower() == 'u')
].apply(lambda x: x[0:re.search('2[\s_-]*', x).end()])

music_df.loc[
    music_df['band_abbrev_autoparsed'].apply(lambda x: re.search('u2', x.lower())).notnull()
    , 'postbandstuff'
] = music_df['stem'][
    music_df['band_abbrev_autoparsed'].apply(lambda x: re.search('u2', x.lower())).notnull()
].apply(lambda x: x[re.search('2[\s_-]*', x).end():len(x)])

         
music_df_bk = music_df

print('ok')

In [ ]:
music_df = music_df_bk
#                    ].dropna().drop_duplicates().sort_values()[0:30]#.values.tolist()
bandendspacers = re.compile('[\W_]+$')

music_df['hasbandendspacers'] = True
music_df.loc[music_df['band_abbrev_autoparsed'].apply(lambda x: re.search(bandendspacers, x)).isnull(), 'hasbandendspacers'] = False

music_df['band_abbrev'] = music_df['band_abbrev_autoparsed']

# Remove trailing spaces from band_abbrev_orig to create band_abbrev_parsed
music_df.loc[
    (music_df['hasbandendspacers']) 
    , 'band_abbrev'] = music_df['band_abbrev_autoparsed'][
    (music_df['hasbandendspacers']) 
].apply(lambda x: x[0:re.search(bandendspacers, x).start()])

# Fix Loreley
music_df.loc[music_df['band_abbrev'].apply(lambda x: 'loreley' in x.lower()), 'band_abbrev'
            ] = music_df['band_abbrev'][music_df['band_abbrev'
                                                ].apply(lambda x: 'loreley' in x.lower())
                                       ].apply(lambda x: x[0:len(x)-7])
music_df_bk = music_df
print('ok')

In [ ]:
# NEED TO FIX THESE BAND VALUES
#music_df['band_abbrev'][~music_df.index.isin(fixbandlist)
#                              ].dropna().drop_duplicates().sort_values()[775:]
# VALUES FOUND THAT WAY ARE LISTED IN fixbandist
fixbandlist = []
fixbandlist += ['567194.html', '126863.html', '566104.html', '566727.html', '566252.html', '567047.html']
fixbandlist += ['566973.html', '566208.html', '565954.html', '565868.html', '566685.html', '565793.html']
fixbandlist += ['566439.html', '565913.html', '0140111caca68f331e53d9651a3eadf6b7b0f999']
fixbandlist += ['566783.html', '567544.html', '566278.html', '566234.html', '566526.html', '566814.html']
fixbandlist += ['566136.html', '566798.html', '567140.html', '565934.html', '566810.html', '566896.html']
fixbandlist += ['565650.html', '566756.html', '566114.html', '567536.html', '566329.html', '567879.html']
fixbandlist += ['567120.html', '567402.html', '565891.html', '567592.html', '566264.html', '566219.html']
fixbandlist += ['566734.html', '587888.html', '567758.html', '567795.html', '127033.html', '566511.html']
fixbandlist += ['565961.html', '566789.html', '565503.html', '566127.html', '566155.html', '565852.html']
fixbandlist += ['127034.html', '566231.html', '566157.html', '566144.html', '566657.html', '567318.html']
fixbandlist += ['565923.html', '566150.html', '566066.html', '565901.html']
fixbandlist += ['567771.html', 'c2b7c10aa42aa6f55c8537ece72ed5726ae6b1b3', '566903.html']
fixbandlist += ['567234.html', '566050.html', '567202.html', '567760.html', '587769.html', '567336.html']
# Write the rows that need to be fixed into a CSV, fix them in XLSX document
#music_df[['band_abbrev']][music_df.index.isin(fixbandlist)].sort_values(by='band_abbrev').to_csv('bandstofix.csv')


# Load the uploaded Excel document with the fixes
fixed_bands_df = pandas.read_excel('bandstofix.xlsx', index_col='ID')

# Perform band fixes
music_df = music_df.join(fixed_bands_df, how='left')
music_df.loc[music_df['band_abbrev_fixed'].notnull(), 'band_abbrev'] = music_df['band_abbrev_fixed']
music_df = music_df.drop('band_abbrev_fixed', axis=1)

# Find blank values of band_abbrev that can be found from stem values and fixed
#music_df['stem'][
#    (~music_df['songonly']) &
#    (music_df['band_abbrev'] == '')
#].sort_values()[775:800]
addbandlist = []
addbandlist += ['566895.html', '566989.html', '567507.html', '567718.html', '566324.html', '566687.html']
addbandlist += ['567464.html', '567692.html', '565610.html', '566238.html', '565714.html', '566937.html']
addbandlist += ['566938.html', '567634.html', '566723.html', '03bcddfb8e4c6e4df25f285517205919e41bca35']
addbandlist += ['127059.html', '566930.html', '566932.html', '566655.html', '565529.html', '567192.html']
addbandlist += ['567503.html', '565750.html', '566261.html', '567810.html', '565970.html', '567294.html']
addbandlist += ['566280.html', '566679.html', 'f704e684bd9817702d272c0d06cd118d695b0517', '565958.html']
addbandlist += ['567845.html', '566294.html', '566673.html', '566674.html', '566805.html', '566804.html']
addbandlist += ['566854.html', '566726.html', '566748.html', '566793.html', '566873.html', '566485.html']
addbandlist += ['565929.html', '567431.html', '565966.html', '566629.html']
addbandlist += ['567201.html', '567025.html', '567022.html', '567150.html', '567088.html']
addbandlist += ['567285.html', '565948.html', '566371.html', '566119.html', '567146.html', '566724.html']
addbandlist += ['566879.html', '567566.html', '567368.html', '567271.html']
addbandlist += ['126776.html', '127051.html', '567639.html', '567323.html', '126935.html', '127156.html']
addbandlist += ['566933.html', '567830.html', '565959.html', '566017.html', '567417.html']
addbandlist += ['567653.html', '566851.html', '567233.html', '567496.html', '566488.html']
addbandlist += ['566106.html', 'f984d4ab17b0a85de1bfa9ffab80efd7625b8122', '567338.html']
addbandlist += ['567717.html', '567578.html', '566890.html', '566546.html', '565723.html']
addbandlist += ['3c39e32bf0918ec3190ecda08442eba0d8b2faa8', '567064.html', '566799.html', '567275.html']
addbandlist += ['567442.html', '587360.html', '565748.html', '587570.html', '587699.html']
addbandlist += ['587637.html', '566375.html', '587700.html', '587634.html', '587693.html']
addbandlist += ['587632.html', '126991.html', '126990.html', '567166.html', '127071.html']
addbandlist += ['567240.html', '567775.html', '567053.html', '565602.html']
music_df['stem'][music_df.index.isin(addbandlist)].sort_values().to_csv('bandstoadd.csv')

music_df_bk = music_df
print('ok')

In [ ]:
music_df = music_df_bk

# Load the uploaded Excel document with the fixes
added_bands_df = pandas.read_excel('bandstoadd.xlsx', index_col='ID')

# Perform band fixes
#music_df = music_df.join(added_bands_df, how='left')
#music_df.loc[music_df['band_abbrev_new'].notnull(), 'band_abbrev'] = music_df['band_abbrev_new']
#music_df = music_df.drop('band_abbrev_new', axis=1)

# Lower and strip band named parsed
#music_df['band_abbrev'] = music_df['band_abbrev'].apply(lambda x: x.lower().strip())

#music_df_bk = music_df
music_df = music_df.join(added_bands_df, how='left')

music_df.loc[music_df['band_abbrev_new'].notnull(), 'band_abbrev'] = music_df['band_abbrev_new']

music_df = music_df.drop(['stem_old', 'band_abbrev_new'], axis=1)

music_df['band_abbrev'] = music_df['band_abbrev'].apply(lambda x: x.lower().strip())

#music_df.sample(3)
music_df_bk = music_df
print('ok')

In [ ]:
# GET LIST OF OFFICIAL ABBREVIATIONS

print('Retrieving from etree')
pageurl = 'http://wiki.etree.org/index.php?page=BandAbbreviations'
page = urllib.request.urlopen(pageurl).read()
soup = bs(page, "html.parser")
tables = soup.find_all('table')

#for thistable in tables:
#    print(thistable)
#    print('---------')

bandtable = tables[1]
bands = bandtable.find_all('tr')
allbands = []

print('Thinking...')
for thisband in bands[1:]:
    theirdata = thisband.find_all('td')
    oneband = {}
    oneband['name'] = theirdata[0].text
    oneband['abbrev'] = theirdata[1].text
    try:
        oneband['otherabbrev'] = theirdata[2].text
    except:
        pass
    try:
        oneband['notes'] = theirdata[3].text
    except:
        pass
    allbands.append(oneband)
abbrev_df = pandas.DataFrame.from_records(allbands)
abbrev_df.index.name = 'rownumber'

column_order = ['name', 'abbrev', 'otherabbrev', 'notes']
abbrev_df = abbrev_df[column_order]
abbrev_df = abbrev_df.fillna('')

# Trim one space from beginning and end of band name
abbrev_df['name'] = abbrev_df['name'].apply(lambda x: str(x).strip())

# Trim away brackets from beginning and end of band name (for some bands)
abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[0])) == '['), 
                             abbrev_df['name'].apply(lambda x: str(x)[1:]), 
                             abbrev_df['name']) 

abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[len(str(x))-1])) == ']'),
                              abbrev_df['name'].apply(lambda x: str(x)[:-1]), 
                              abbrev_df['name'])

abbrev_df['abbrev'] = abbrev_df['abbrev'].apply(lambda x: str(x).lower().strip())
abbrev_df['otherabbrev'] = abbrev_df['otherabbrev'].apply(lambda x: str(x).lower().strip())

#abbrev_df['abbrev'].apply(lambda x: '|'+str(x)+'|')

abbrev_df.to_csv('abbreviations.csv')
os.listdir()
#print('ok')
#pprint(abbrev_df.values.tolist())
canonical_abbrevs = abbrev_df['abbrev'][abbrev_df['abbrev'] != ''].drop_duplicates().tolist()
#canonical_abbrevs

# Get variant abbreviations
variant_abbrevs = []
for x in abbrev_df['otherabbrev'].drop_duplicates():
    if (',' in x):
        for y in x.split(','):
            variant_abbrevs.append(y.strip())
    else:
        if (x.strip() != ''):
            variant_abbrevs.append(x.strip())

variant_abbrevs = sorted(variant_abbrevs)

# Fix wilco by moving it from variant to canonical
canonical_abbrevs.append('wilco')
variant_abbrevs.remove('wilco')

# Strip and lower
canonical_abbrevs = [x.lower().strip() for x in canonical_abbrevs]    
variant_abbrevs = [x.lower().strip() for x in variant_abbrevs]    

print('Parsed {0:,.0f} canonical abbreviations and {1:,.0f} variant abbreviations.'.format(len(canonical_abbrevs), len(variant_abbrevs)))

In [ ]:
music_df = music_df_bk

music_df['abbrev_type'] = 'notfound'
music_df.loc[music_df['band_abbrev'] == '', 'abbrev_type'] = 'blank'
music_df.loc[music_df['band_abbrev'].apply(lambda x: x.lower() in canonical_abbrevs), 'abbrev_type'] = 'canonical'
music_df.loc[music_df['band_abbrev'].apply(lambda x: x.lower() in variant_abbrevs), 'abbrev_type'] = 'variant'

# Band names used incorrect abbreviations
incorrect = ['phish', 'widespreadpanic']
incorrect += ['gov`t mule', 'pl&amp;f']
    
music_df.loc[music_df['band_abbrev'].isin(incorrect), 'abbrev_type'] = 'incorrect'

music_df_bk = music_df

music_df.groupby('abbrev_type').size()

In [ ]:
music_df = music_df_bk
#music_df[['band_abbrev']][music_df['abbrev_type'] == 'notfound'].drop_duplicates().sort_values(by='band_abbrev').to_csv('bands_to_verify.csv')

verified_bands_df = pandas.read_excel('bands_verified_anew.xlsx', index_col='ID')
music_df = music_df.reset_index().merge(verified_bands_df, how='left', on='band_abbrev').set_index('ID')

music_df.loc[(music_df['abbrev_type'] == 'notfound') & (music_df['band_or_not'] == 'yes'), 'abbrev_type'] = 'assigned'
music_df.loc[(music_df['abbrev_type'] == 'notfound') & (music_df['band_or_not'] == 'maybe'), 'abbrev_type'] = 'maybeband'
music_df.loc[(music_df['abbrev_type'] == 'notfound') & (music_df['band_or_not'] == 'no'), 'abbrev_type'] = 'notband'

#music_df = music_df.drop(['band_abbrev_r', 'band_or_not'], axis=1)

music_df.groupby('abbrev_type').size()

In [ ]:
music_df[music_df['abbrev_type'] == 'maybeband']